# Analysis

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import yt
from rich import print
from icecream import ic
import panel as pn
from loguru import logger

import logging
import json
from yt.data_objects.time_series import SimulationTimeSeries
from yt.data_objects.static_output import Dataset
from matplotlib.pyplot import Axes

In [21]:
from utils.plot import *

In [4]:
dim = 1
# dim = 2
# dim = 3
beta = 0.25
theta = 60.0
plasma_resistivity = 100.0

In [5]:
import os
from pathlib import Path

try:
    base_dir = Path(os.getcwd()) / "01_oblique_linear_alfven"
    sub_dir = f"dim_{dim}_beta_{beta}_theta_{theta}_eta_{plasma_resistivity}"
    directory = base_dir / sub_dir
    os.chdir(directory)
    os.makedirs("figures", exist_ok=True)
except FileNotFoundError:
    pass

In [6]:
pn.extension()

# load simulation parameters
with open("sim_parameters.json", "rb") as f:
    meta = json.load(f)

In [7]:
ps = "ions"

## Fields

In [8]:
from utils import load_ts_all
ts_field, ts_part = load_ts_all(meta) 

In [9]:
ic(len(ts_field))
ds_field: Dataset = ts_field[0]
ds_part: Dataset = ts_part[0]

ic| len(ts_field): 501


## Particles

In [66]:
# plot_field_with_plasma_profile_ts(ts_field, ts_part, name="v_comp", step=4)

In [67]:
#: check the field list
# print(ds_part.field_list)
# print(ds_part.derived_field_list)

### Pressure

In [68]:
import polars as pl
import polars.selectors as cs

In [69]:
# df["x"] = pd.qcut(df["particle_position_x"], q=10)

In [70]:
from utils import ds2df

n = 64
if dim == 1 or dim == 2:
    i = 0
else:
    i = 2

In [71]:
from utils import ds2xr, ds2df

### yt

In [ ]:
ts: SimulationTimeSeries = yt.load('diags/diag1??????')
# ts = yt.load('./diags/diag???0032')

In [ ]:
def plot(ds, normalize = True, **kwargs):
    ad = ds.all_data()
    fields = ["Bx", "By", "Bz"]

    match meta["dim"]:
        case 1: pos = "x"
        case 2: pos = "y"
        
    pos = ad[pos]
    
    if normalize:
        pos = pos / meta['d_i']
    
    for field in fields:
        plt.plot(pos, ad[field], label=field, **kwargs)
    
    plt.xlabel("x ($d_i$)")
        
def hodogram(ds, comp1="By", comp2="Bz"):
    time = ds.current_time
    time_norm = time.value / meta['t_ci']
    ad = ds.all_data()
    plt.plot(ad[comp1], ad[comp2], label=f"t={time_norm:.2f}")
    plt.xlabel(comp1)
    plt.ylabel(comp2)

In [ ]:
for i, ds in enumerate(ts):
    alpha = (i + 1) / (len(ts)+1)
    plot(ds, alpha=alpha)
    plt.show()  # Show each plot separately

In [ ]:
i = 4
_ts = ts[0:i]
for i, ds in enumerate(_ts):
    alpha = (i + 1) / (len(_ts)+1)
    plot(ds, alpha=alpha)
    plt.show()  # Show each plot separately

In [ ]:
for ds in ts:
    hodogram(ds)
    plt.legend()

In [ ]:
i = 5
for ds in ts[0:i]:
    hodogram(ds)
    plt.legend()

In [ ]:
yt.SlicePlot(ds, "z", ("boxlib", "Bz"))

In [ ]:
ds.all_data()

In [ ]:
fields = [
    ("Bx"),
    ("By"),
    ("Bz"),
    ("mesh", "magnetic_field_strength"),
]

In [ ]:
for ds in ts.piter():
    p = yt.plot_2d(ds, fields=fields)
    p.set_log(fields, False)
    fig = p.export_to_mpl_figure((2, 2))
    fig.tight_layout()
    fig.savefig(f"figures/{ds}_magnetic_field.png")

### Average magnetic field

In [35]:
def plot_avg(ds):
    fields = [
        ("Bx"),
        ("By"),
        ("Bz"),
    ]

    ad = ds.all_data()
    df = ad.to_dataframe(fields + ["x", zaxis])
    # compute the magnetic field strength
    df = df.assign(B=lambda x: (x.Bx**2 + x.By**2 + x.Bz**2) ** 0.5)

    axes = df.groupby(zaxis).mean().plot(y=fields + ["B"], subplots=True)
    return axes[0].figure

In [36]:
def plot_avg_ts(i):
    return plot_avg(ts[i])
    
time_widget = pn.widgets.IntSlider(name="Time", value=1, start=0, end=len(ts)-1)
bound_plot = pn.bind(plot_avg_ts, i=time_widget)

pn.Column(time_widget, bound_plot)

NameError: name 'ts' is not defined

In [ ]:
for ds in ts.piter():
    plot_avg(ds)

In [ ]:
ds.print_stats()
print(ds.field_list)

In [ ]:
grid = ds.r[:,:,:]
obj = grid.to_xarray(fields=fields)